In [1]:
# Dependenies
import pandas as pd
import numpy as np

In [2]:
# Store filepath in a variable
school_file = "Resources/schools_complete.csv"
student_file = "Resources/students_complete.csv"

In [3]:
# Read data file, show header
schools_df = pd.read_csv(school_file)
students_df = pd.read_csv(student_file)

In [4]:
# Merge two files into one dataframe
complete_df = pd.merge(students_df, schools_df, how = "left", on = ["school_name","school_name"])
#complete_df.head(2)

In [5]:
# Calculate District Totals - Part One
d_SchoolCount = len(complete_df["school_name"].unique())
d_StudentCount = schools_df["size"].sum()
d_BudgetTotal = schools_df["budget"].sum()

d_AvgReadScore = complete_df["reading_score"].mean()
d_AvgMathScore = complete_df["math_score"].mean()
#d_SchoolCount
#d_StudentCount

In [6]:
# calculate math passing score
mathdf = students_df.loc[students_df["math_score"]>=70]
percent_pass_m = mathdf["student_name"].count()/d_StudentCount*100
#print(percent_pass_m)

# calculate reading passing score
readingdf = students_df.loc[students_df["reading_score"]>=70]
percent_pass_r = readingdf["student_name"].count()/d_StudentCount*100
#print(percent_pass_r)

In [8]:
# Calculating Percent Passing Scores:
overall_pass_rate = ((d_AvgReadScore + d_AvgMathScore)/2)
#print(overall_pass_rate)

In [10]:
#Create District Summary Table
district_summary_table = pd.DataFrame({"Total Schools": [d_SchoolCount],
                              "Total Students": [d_StudentCount],
                              "Total Budget": [d_BudgetTotal],
                             "Avg Math Score": [d_AvgMathScore],
                             "Avg Reading Score": [d_AvgReadScore],
                             "% Passing Math": [percent_pass_m],
                             "% Passing Reading": [percent_pass_r],
                             "Overall Passing Rate": [overall_pass_rate]
                             })
#district_summary_table

In [11]:
#Format Total Budget
district_summary_table["Total Budget"] = district_summary_table["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary_table

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


In [12]:
#Use the groupby function to group by school name, and the agg function to get other values
# Create new df
schools_df1 = complete_df.groupby("school_name").agg(
        total_students = pd.NamedAgg(column="student_name", aggfunc='count'),
        average_math_score = pd.NamedAgg(column="math_score", aggfunc='mean'),
        average_reading_score = pd.NamedAgg(column="reading_score", aggfunc='mean'))
#schools_df1.head(2)

In [13]:
# Get list of schools
schools_list = schools_df["school_name"].unique()
#schools_list

#Use .loc command to create df with the school type and budget
schools_df2 = schools_df.loc[schools_df["school_name"] == schools_list, ["school_name", "type", "budget"]]
#schools_df2.head(2)

In [14]:
# Merge school_df1 + school_df2
school_summ1 = pd.merge(schools_df1, schools_df2, on="school_name")
#school_summ1.head(2)

In [15]:
# create new column for 'budget per student' and add to school_summ1 df
school_summ1["budget_per_student"] = school_summ1["budget"] / school_summ1["total_students"]
#school_summ1.head(2)

In [16]:
#Create Pass/Fail columns.  Select only records of those that passed, then count row total
columnsM = ["school_name", "math_score"]
columnsR = ["school_name", "reading_score"]

#get count of students passing math test
studentsPassM = students_df.loc[students_df["math_score"] >= 70, columnsM]
studentsPassMath = studentsPassM.groupby(["school_name"]).count()

#get count of students passing reading test
studentsPassR = students_df.loc[students_df["reading_score"] >= 70, columnsR]
studentsPassReading = studentsPassR.groupby(["school_name"]).count()

In [17]:
#Merge files for passing scores, add column for overall passing rate
school_summ2 = pd.merge(studentsPassMath, studentsPassReading, on="school_name")
#school_summ2.head(2)

In [18]:
# merge data for # students per school
school_summ3 = pd.merge(school_summ2, school_summ1, on="school_name")
school_summ3["Percent Passing Math"] = school_summ3["math_score"] / school_summ3["total_students"] * 100
school_summ3["Percent Passing Reading"] = school_summ3["reading_score"] / school_summ3["total_students"] * 100
#school_summ3.head(2)

In [19]:
# Create new column for the overall passing rate
school_summ3["Overall Passing Rate"] = ((school_summ3["Percent Passing Math"] + school_summ3["Percent Passing Reading"])/2)
#school_summ3.head(2)

In [20]:
# rename and format remaining columns
school_summ3 = school_summ3.rename(columns = {"school_name":"School Name", "type":"School Type",
                                              "total_students":"Total Students",
                                             "budget":"Total School Budget",
                                             "average_math_score":"Average Math Score",
                                             "average_reading_score":"Average Reading Score",
                                             "budget_per_student":"Budget Per Student"})

school_summ3["Total School Budget"] = school_summ3["Total School Budget"].astype(float).map("${:,.2f}".format)
school_summ3["Budget Per Student"] = school_summ3["Budget Per Student"].astype(float).map("${:,.2f}".format)
#school_summ3.head(3)

In [21]:
#Create Schools Summary Table
schools_summary_table = school_summ3[["School Name", "School Type","Total Students", 
                                   "Total School Budget", "Budget Per Student", 
                                   "Average Math Score", "Average Reading Score",
                                   "Percent Passing Math", "Percent Passing Reading", "Overall Passing Rate"]]
schools_summary_table

,School Name,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


In [22]:
# Sort and Display the top five schools in overall passing rate
TopFiveSchools_by_PassingRate = schools_summary_table.sort_values("Overall Passing Rate", ascending=False)
TopFiveSchools_by_PassingRate.head(5)

,School Name,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
12,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
13,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [23]:
# Sort and Display the bottom five schools in overall passing rate
TopFiveSchools_by_PassingRate = schools_summary_table.sort_values("Overall Passing Rate", ascending=True)
TopFiveSchools_by_PassingRate.head(5)

,School Name,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
10,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
7,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
3,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [24]:
#Math scores by grade: .loc by grade, then groupby school and put in a Summary Table
grade9math = students_df.loc[students_df["grade"] == "9th", ["school_name", "math_score"]]
grade9M = grade9math.groupby("school_name").mean()

grade10math = students_df.loc[students_df["grade"] == "10th", ["school_name", "math_score"]]
grade10M = grade10math.groupby("school_name").mean()

math_merge = pd.merge(grade9M, grade10M, on="school_name", suffixes=("_9th", "_10th"))
Math_by_Grade_Table = pd.DataFrame(math_merge)

Math_by_Grade_Table = Math_by_Grade_Table.rename(columns={"math_score_9th":"9th", "math_score_10th":"10th"})

grade11math = students_df.loc[students_df["grade"] == "11th", ["school_name", "math_score"]]
grade11M = grade11math.groupby("school_name").mean()

Math_by_Grade_Table = pd.merge(Math_by_Grade_Table, grade11M, on="school_name")
Math_by_Grade_Table = Math_by_Grade_Table.rename(columns={"math_score":"11th"})

grade12math = students_df.loc[students_df["grade"] == "12th", ["school_name", "math_score"]]
grade12M = grade12math.groupby("school_name").mean()

Math_by_Grade_Table = pd.merge(Math_by_Grade_Table, grade12M, on="school_name")
Math_by_Grade_Table = Math_by_Grade_Table.rename(columns={"math_score":"12th"})

Math_by_Grade_Table

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [25]:
#Reading scores by grade: .loc by grade, then groupby school and put in a Summary Table
grade9reading = students_df.loc[students_df["grade"] == "9th", ["school_name", "reading_score"]]
grade9R = grade9reading.groupby("school_name").mean()

grade10reading = students_df.loc[students_df["grade"] == "10th", ["school_name", "reading_score"]]
grade10R = grade10reading.groupby("school_name").mean()

reading_merge = pd.merge(grade9R, grade10R, on="school_name", suffixes=("_9th", "_10th"))
Reading_by_Grade_Table = pd.DataFrame(reading_merge)

Reading_by_Grade_Table = Reading_by_Grade_Table.rename(columns={"reading_score_9th":"9th", "reading_score_10th":"10th"})

grade11reading = students_df.loc[students_df["grade"] == "11th", ["school_name", "reading_score"]]
grade11R = grade11reading.groupby("school_name").mean()

Reading_by_Grade_Table = pd.merge(Reading_by_Grade_Table, grade11R, on="school_name")
Reading_by_Grade_Table = Reading_by_Grade_Table.rename(columns={"reading_score":"11th"})

grade12reading = students_df.loc[students_df["grade"] == "12th", ["school_name", "reading_score"]]
grade12R = grade12reading.groupby("school_name").mean()

Reading_by_Grade_Table = pd.merge(Reading_by_Grade_Table, grade12R, on="school_name")
Reading_by_Grade_Table = Reading_by_Grade_Table.rename(columns={"reading_score":"12th"})

Reading_by_Grade_Table

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [26]:
#Create bins for scores by school spending
spending_bins = [0, 585, 630, 645, 655]
group_names = ["< $585", "$585 - $630", "$630 - $645", "$645 - $655"]

In [27]:
# Slice the data and put into bins, and add Spending Range column to the end of the table
school_summ1["Spending Range"] = pd.cut(school_summ1["budget_per_student"], spending_bins, labels=group_names)
#school_summ1.head(2)

In [28]:
# Create a new Spending Table to grab school and range from summ1 df.
SpendingTable = school_summ1
SpendingTable = SpendingTable[["school_name","Spending Range"]]
SpendingTable = SpendingTable.rename(columns={"school_name":"School Name"})
#SpendingTable.head(5)

In [29]:
# Create new Spending Table 2 to bring forward values from summ3 table to prep for data merge
SpendingTable2 = school_summ3[["School Name","Average Math Score","Average Reading Score",
                               "Percent Passing Math","Percent Passing Reading", "Overall Passing Rate"]]
#SpendingTable2.head(5)

In [30]:
# Merge previous two Spending Tables into one new SpendingRangeTable
SpendingRangeTable = pd.merge(SpendingTable2, SpendingTable, on="School Name")
#SpendingRangeTable.head(3)

In [31]:
# Group the SpendingRangeTable by Spending Range and provide the new averages within these ranges.
SpendingGroup=SpendingRangeTable.groupby("Spending Range")
SpendingGroup[["Average Math Score","Average Reading Score","Percent Passing Math",
               "Percent Passing Reading", "Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Spending Range,,,,,
< $585,83.455399,83.933814,93.460096,96.610877,95.035486
$585 - $630,81.899826,83.155286,87.133538,92.718205,89.925871
$630 - $645,78.518855,81.624473,73.484209,84.391793,78.938001
$645 - $655,76.997210,81.027843,66.164813,81.133951,73.649382


In [32]:
# Create bins for scores by school size
size_bins = [0, 1000, 2000, 3500, 5000]
group_names = ["Very Small ( < 1000)", "Small (1000 - 2000)", "Medium (2000 - 3500)", "Large(3500 - 5000)"]

In [33]:
# Create output file for binned school file
school_summ1["School Size"] = pd.cut(school_summ1["total_students"], size_bins, labels=group_names)
#school_summ1.head()

In [34]:
#Create new SizeTable, with only necessary columns, to prep for merge
size = school_summ1[["school_name","School Size"]]
size = size.rename(columns={"school_name":"School Name"})
#size.head(2)

In [35]:
# Merge SizeTable with SpendingRangeTAble to get other desired columns
SizeTable = pd.merge(SpendingTable2, size, on=("School Name"))
#SizeTable.head(2)

In [36]:
# Group the SizeTable by School Size and provide the new averages within these ranges.
SizeGroup = SizeTable.groupby("School Size")
SizeGroup[["Average Math Score","Average Reading Score","Percent Passing Math",
               "Percent Passing Reading", "Overall Passing Rate"]].mean()

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Size,,,,,
Very Small ( < 1000),83.821598,83.929843,93.550225,96.099437,94.824831
Small (1000 - 2000),83.374684,83.864438,93.599695,96.790680,95.195187
Medium (2000 - 3500),78.429493,81.769122,73.462428,84.473577,78.968003
Large(3500 - 5000),77.063340,80.919864,66.464293,81.059691,73.761992


In [37]:
# Get the schools list with type from earlier step
school_type = schools_df2[["school_name","type"]]
school_type = school_type.rename(columns={"school_name":"School Name","type":"School Type"})
#school_type.head()

In [38]:
#Merge the school type info with the earlier summary table
TypeTable = pd.merge(SpendingTable2, school_type, on="School Name")
#TypeTable.head(2)

In [39]:
SummaryTypeTable = TypeTable.groupby("School Type").mean()
SummaryTypeTable

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [40]:
#Create my own summary table
My_summary_table = schools_summary_table[["School Name","School Type","Total Students", "Budget Per Student","Overall Passing Rate"]]
My_summary_table = My_summary_table.sort_values(["School Type","Total Students"], ascending=True)
My_summary_table

,School Name,School Type,Total Students,Budget Per Student,Overall Passing Rate
6,Holden High School,Charter,427,$581.00,94.379391
9,Pena High School,Charter,962,$609.00,95.270270
4,Griffin High School,Charter,1468,$625.00,95.265668
12,Thomas High School,Charter,1635,$638.00,95.290520
11,Shelton High School,Charter,1761,$600.00,94.860875
14,Wright High School,Charter,1800,$583.00,94.972222
1,Cabrera High School,Charter,1858,$582.00,95.586652
13,Wilson High School,Charter,2283,$578.00,95.203679
3,Ford High School,District,2739,$644.00,73.804308
7,Huang High School,District,2917,$655.00,73.500171
